In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv001.ib.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/notebooks


In [2]:
import sys, os
import numpy as np

sys.path.append('..')
%aimport mre_pinn

sys.path.append('../../param_search')
%aimport param_search
ps = param_search

Using backend: pytorch



# Helmholtz equation

## Effect of PDE domain sampling method

In this experiment I want to understand the PDE dynamics on a simple 1D problem. I have observed a strange Gibbs-like phenomenon emerge when training using the Helmholtz equation and I have a few hypotheses. One is that this is due to violation of the homogeneity assumption when we do pseudorandom sampling of the PDE domain. If we instead train only wrt the data points, this should reduce the assumption violation. Another possibility is that it is a result of the spectral bias of the model, as it relates to the activation functions. So for this I test using different activation functions and different multipliers on the first sine activation function.

In [55]:
# define the job template and name format

template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=bio170034p
#SBATCH --partition=BatComputer
#SBATCH --gres=gpu:rtx5000:1
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
source activate MRE-PINN

python ../../../train.py \\
    --data_root ../../../data/BIOQIC \\
    --data_name fem_box \\
    --frequency {frequency} \\
    --xyz_slice {xyz_slice} \\
    --pde_name {pde_name} \\
    --omega0 {omega0} \\
    --n_layers {n_layers} \\
    --n_hidden {n_hidden} \\
    --activ_fn {activ_fn} \\
    --learning_rate {learning_rate} \\
    --pde_loss_wt {pde_loss_wt} \\
    --data_loss_wt {data_loss_wt} \\
    --batch_size {batch_size} \\
    --n_domain {n_domain} \\
    --n_iters {n_iters} \\
    --test_every {test_every} \\
    --save_every {save_every} \\
    --save_prefix {job_name}    
'''
name = 'train_{frequency}_{xyz_slice}_{pde_name}_{omega0}_{activ_fn}_{pde_distrib}'

# define the parameter space

param_space = ps.ParamSpace(
    frequency=[80],
    xyz_slice=['1D'],
    pde_name=['helmholtz', 'hetero'],
    omega0=[4, 8, 16],
    n_layers=5,
    n_hidden=16,
    activ_fn=['s', 't'],
    learning_rate=1e-3,
    pde_loss_wt=1,
    data_loss_wt=1e-8,
    batch_size=80,
    n_domain=128-80,
    pde_distrib=['pseudo', 'sobol', 'uniform'],
    n_iters=25000,
    test_every=1000,
    save_every=1000
)

len(param_space)

36

In [56]:
%autoreload
expt_name = '2022-07-24'
jobs = ps.submit(template, name, param_space, work_dir=expt_name, verbose=True)
jobs.to_csv(f'{expt_name}.jobs')

100%|██████████| 36/36 [00:03<00:00, 11.65it/s]
[9951620, 9951621, 9951622, 9951623, 9951624, 9951625, 9951626, 9951627, 9951628, 9951629, 9951630, 9951631, 9951632, 9951633, 9951634, 9951635, 9951636, 9951637, 9951638, 9951639, 9951640, 9951641, 9951642, 9951643, 9951644, 9951645, 9951646, 9951647, 9951648, 9951649, 9951650, 9951651, 9951652, 9951653, 9951654, 9951655]


In [58]:
status_cols = ['job_name', 'job_state', 'node_id', 'runtime', 'stdout', 'stderr']
ps.status(jobs)[status_cols]

,job_name,job_state,node_id,runtime,stdout,stderr
job_id,,,,,,
9951620,train_80_1D_helmholtz_4_s_pseudo,RUNNING,dv001,1:20,NaN,NaN
9951621,train_80_1D_helmholtz_4_s_sobol,RUNNING,dv001,1:00,NaN,NaN
9951622,train_80_1D_helmholtz_4_s_uniform,RUNNING,dv001,1:00,NaN,NaN
9951623,train_80_1D_helmholtz_4_t_pseudo,RUNNING,dv001,1:00,NaN,NaN
9951624,train_80_1D_helmholtz_4_t_sobol,RUNNING,dv002,1:00,NaN,NaN
9951625,train_80_1D_helmholtz_4_t_uniform,RUNNING,dv002,1:00,NaN,NaN
9951626,train_80_1D_helmholtz_8_s_pseudo,RUNNING,dv002,1:00,NaN,NaN
9951627,train_80_1D_helmholtz_8_s_sobol,RUNNING,dv002,1:00,NaN,NaN
9951628,train_80_1D_helmholtz_8_s_uniform,RUNNING,dv002,1:00,NaN,NaN
